In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from data.dataloader import Covid19IndiaLoader, JHULoader

In [ ]:
df_vaccine = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv')
df_vaccine['date'] = pd.to_datetime(df_vaccine['date'])

In [ ]:
obj = JHULoader()
dataframes = obj.pull_dataframes_cached()

In [ ]:
df_cases = dataframes['df_global']
df_cases.rename({
    'ConfirmedCases': 'total',
    'ActiveCases': 'active',
    'RecoveredCases': 'recovered',
    'Deaths': 'deceased',
    'Date': 'date'
}, inplace=True, axis=1)

In [ ]:
COUNTRY = 'Russia'

df_vaccine_region = df_vaccine[df_vaccine['location'] == COUNTRY]
df_cases_region = df_cases[(df_cases['Country/Region'] == COUNTRY) & (df_cases['Province/State'].isna())]
df_cases_region.loc[:, 'daily'] = df_cases_region.loc[:, 'total'].diff()
df_cases_region_trunc = df_cases_region[df_cases_region['date'].isin(df_vaccine_region['date'])]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(df_vaccine_region['date'], df_vaccine_region['people_vaccinated_per_hundred'], label='Pop(%) w at least 1 dose')
ax.plot(df_vaccine_region['date'], df_vaccine_region['people_fully_vaccinated_per_hundred'], c='C2', label='Pop(%) w 2 doses')

ax_twin = ax.twinx()
df_cases_region_trunc
# ax_twin.plot(df_vaccine_region['date'], df_cases_region_trunc['daily'], ls='--', label='Daily cases', c='red')
ax_twin.plot(df_vaccine_region['date'], df_cases_region_trunc['daily'].rolling(7).mean(), label='Daily cases', c='red')
ax.legend()
ax_twin.legend()
# ax.grid()